# Create z-score files for CREDIT

In [1]:
import os
import yaml
import numpy as np
import xarray as xr

## File creation

### 6 hourly mean std files

In [2]:
# get variable information from data_preprocessing/config
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [3]:
N_levels = 16
base_dir = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg_stage1/'

In [4]:
# get variable names
varnames = list(conf['zscore'].keys())
varnames = varnames[:-3] # remove save_loc and others

varname_surf = list(set(varnames) - set(['U', 'V', 'T', 'Q']))
varname_upper = ['U', 'V', 'T', 'Q']

# collect computed mean and variance values
# See "qsub_STEP01_compute_mean_std.ipynb"
MEAN_values = {}
STD_values = {}

for varname in varname_surf:
    save_name = conf['zscore']['save_loc'] + '{}_mean_std_{}.npy'.format(conf['zscore']['prefix'], varname)
    mean_std = np.load(save_name)
    MEAN_values[varname] = mean_std[0]
    STD_values[varname] = mean_std[1]

for varname in varname_upper:

    # -------------------------------------------- #
    # allocate all levels
    mean_std_all_levels = np.empty((2, N_levels))
    mean_std_all_levels[...] = np.nan
    
    for i_level in range(N_levels):
        save_name = conf['zscore']['save_loc'] + '{}_level{}_mean_std_{}.npy'.format(
            conf['zscore']['prefix'], i_level, varname)
        mean_std = np.load(save_name)
        mean_std_all_levels[:, i_level] = mean_std

    # -------------------------------------------- #
    # save
    MEAN_values[varname] = np.copy(mean_std_all_levels[0, :])
    STD_values[varname] = np.copy(mean_std_all_levels[1, :])

In [5]:
ds_example = xr.open_zarr(base_dir+'all_in_one/ERA5_mlevel_1deg_6h_lev16_1979.zarr')

In [6]:
# ------------------------------------------------------- #
# create xr.DataArray for mean

# Initialize level coord
level = np.array(ds_example['level'])

# Initialize dataset
ds_mean_6h = xr.Dataset(coords={"level": level})

for varname, data in MEAN_values.items():
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_mean_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_mean_6h[varname] = data_array

In [7]:
# ------------------------------------------------------- #
# create xr.DataArray for std

# use the same level coord as mean
ds_std_6h = xr.Dataset(coords={"level": level})

for varname, data in STD_values.items():
    data = np.sqrt(data)
    if len(data.shape) == 1:
        data_array = xr.DataArray(
            data,
            dims=["level",],
            coords={"level": level},
            name=varname,
        )
        ds_std_6h[varname] = data_array
    else:
        data_array = xr.DataArray(
            data,
            name=varname,
        )
        ds_std_6h[varname] = data_array

In [8]:
# ------------------------------------------------------- #
# Same to netCDF
ds_mean_6h.to_netcdf(base_dir+'mean_std/mean_6h_1979_2018_1deg.nc')
ds_std_6h.to_netcdf(base_dir+'mean_std/std_6h_1979_2018_1deg.nc')

In [9]:
    # mean_path: '/glade/derecho/scratch/ksha/CREDIT_data/mean_6h_1979_2018_16lev_0.25deg.nc'
    # # regular z-score version
    # # std_path: '/glade/derecho/scratch/ksha/CREDIT_data/std_6h_1979_2018_16lev_0.25deg.nc'
    # # residual norm version
    # std_path: '/glade/derecho/scratch/ksha/CREDIT_data/std_residual_6h_1979_2018_16lev_0.25deg.nc'

In [12]:
base_dir+'mean_std/std_6h_1979_2018_1deg.nc'

'/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg_stage1/mean_std/std_6h_1979_2018_1deg.nc'

In [10]:
# ------------------------------------------------------- #
# Compare with my old ones
STD_conserve = xr.open_dataset(base_dir+'mean_std/std_6h_1979_2018_1deg.nc')
STD_bilinear = xr.open_dataset('/glade/derecho/scratch/ksha/CREDIT_data/std_6h_1979_2018_16lev_0.25deg.nc')

for varname in varnames:
    print('=============== {} ================='.format(varname))
    print(np.array(STD_conserve[varname]))
    print(np.array(STD_bilinear[varname]))

=============== Q =================
[2.88366821e-07 3.43996775e-07 3.02545878e-07 3.72776741e-07
 5.62630432e-07 6.33140064e-06 8.56978283e-05 4.79411907e-04
 9.42083743e-04 1.63327822e-03 2.36612487e-03 3.28568142e-03
 4.89955869e-03 5.80935251e-03 5.94272198e-03 6.00623108e-03]
[2.86539110e-07 3.43501184e-07 3.00897636e-07 3.70370665e-07
 5.65687055e-07 6.40576319e-06 8.72308306e-05 4.86562781e-04
 9.55834282e-04 1.65400575e-03 2.39454699e-03 3.32212385e-03
 4.92020484e-03 5.82008968e-03 5.95355206e-03 6.01718454e-03]
=============== Q500 =================
0.0010675011022210237
0.0010784555247599808
=============== SP =================
9597.773243269761
9564.08828465833
=============== T =================
[ 8.30371089 12.61047024 10.80722619 10.56573898 12.62041742  8.19765517
  9.11070813 13.61353943 14.4098643  14.81314237 15.34843726 15.87551619
 17.0509786  19.29923236 20.81507832 21.09224943]
[ 8.28246086 12.54296629 10.73224169 10.51179137 12.60151149  8.16516086
  9.08780623 1